In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/challange

/content/drive/MyDrive/challange


In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
#from format_submission import format_submission
#from pivottablejs import pivot_ui
#import xgboost as xgb
import warnings
warnings.simplefilter(action='ignore')

from sklearn.metrics import mean_absolute_error as MAE
from sklearn.preprocessing import OneHotEncoder
 

In [ ]:
train_station = pd.read_csv("data/train.csv", sep=",")
train_station['date'] = pd.to_datetime(train_station['date'])
train_station['Postcode'] = train_station['Postcode'].astype(str)
train_station.shape


(1851083, 13)

In [ ]:
#creating the data set
dateExist=train_station.date.dt.minute
#ceating dates and taking only the 15min inteval per hour 
all_dates23=pd.date_range("2020-07-03 00:00:00","2021-02-18 23:45:00",
            freq='min')
min24=all_dates23.minute
df_only_date=pd.DataFrame({"date":all_dates23,"min":min24})
df_only_date=df_only_date[df_only_date["min"].isin(dateExist)]
date23=np.repeat(all_dates23,91)#for the 91 stations 

#creating dow
df_only_date['dow']=(df_only_date.date.dt.dayofweek+10).astype("str")
map_rep={ "16":1,"10":2,"11":3,"12":4,"13":5,"14":6,"15":7}
df_only_date["dow"]=(df_only_date["dow"].replace(map_rep)).astype("int")
dow=np.repeat(df_only_date["dow"].values,91)#for the 91 stations 

#creating trend
day24=all_dates23.dayofyear
trend=list(range(27,len(df_only_date.date)+27))
trend=np.repeat(trend,91)#for the 91 stations 

#creating tod
tod=np.tile(list(range(0,96)),len(day24.unique()))
tod=np.repeat(tod,91)#for the 91 stations 

#Creating station
all_station23=train_station.Station.unique()
station23=np.tile(all_station23, len(all_dates23))

#All togther
min23=date23.minute

date_for_missing=pd.DataFrame({"date":date23,"Station":station23,
                               "min":min23})
full_data=date_for_missing[date_for_missing["min"].isin(dateExist)]
#Merge with train station 
new_train_station=pd.merge(full_data[['date','Station']],
                           train_station,on=['date','Station'],how="left")
#adding trend, tod, dow
new_train_station["trend"]=trend
new_train_station["tod"]=tod
new_train_station['dow'] =dow
new_train_station.head()

,date,Station,Available,Charging,Passive,Other,tod,dow,trend,Latitude,Longitude,Postcode,area
0,2020-07-03,FR*V75*EBELI*1*1,3.0,0.0,0.0,0.0,0,6,27,48.855667,2.354089,75004,south
1,2020-07-03,FR*V75*EBELI*10*1,2.0,1.0,0.0,0.0,0,6,27,48.864240,2.397724,75020,east
2,2020-07-03,FR*V75*EBELI*11*1,2.0,1.0,0.0,0.0,0,6,27,48.861016,2.283080,75016,west
3,2020-07-03,FR*V75*EBELI*12*1,3.0,0.0,0.0,0.0,0,6,27,48.824388,2.336857,75014,south
4,2020-07-03,FR*V75*EBELI*13*1,3.0,0.0,0.0,0.0,0,6,27,48.882640,2.335728,75009,north


In [ ]:
#adding area Postcode Longitude and Latitude to the missing values
def fill_miss_we_know(df,target,name):

    dict_of=df.loc[df[name].isnull()==False,[target,name]].groupby(
                                                            [target])[name].apply(set).to_dict()
    dict_of={i:list(dict_of[i])[0]  for i in dict_of}
    
    df[name]=df[target].map(dict_of)

    return df
df=new_train_station.copy()
df=fill_miss_we_know(df,"Station","area")
df=fill_miss_we_know(df,"Station","Postcode")
df=fill_miss_we_know(df,"Station","Longitude")
df=fill_miss_we_know(df,"Station","Latitude")
df

,date,Station,Available,Charging,Passive,Other,tod,dow,trend,Latitude,Longitude,Postcode,area
0,2020-07-03 00:00:00,FR*V75*EBELI*1*1,3.0,0.0,0.0,0.0,0,6,27,48.855667,2.354089,75004,south
1,2020-07-03 00:00:00,FR*V75*EBELI*10*1,2.0,1.0,0.0,0.0,0,6,27,48.864240,2.397724,75020,east
2,2020-07-03 00:00:00,FR*V75*EBELI*11*1,2.0,1.0,0.0,0.0,0,6,27,48.861016,2.283080,75016,west
3,2020-07-03 00:00:00,FR*V75*EBELI*12*1,3.0,0.0,0.0,0.0,0,6,27,48.824388,2.336857,75014,south
4,2020-07-03 00:00:00,FR*V75*EBELI*13*1,3.0,0.0,0.0,0.0,0,6,27,48.882640,2.335728,75009,north
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018011,2021-02-18 23:45:00,FR*V75*EBELI*95*1,0.0,0.0,0.0,3.0,95,5,22202,48.868511,2.295469,75016,west
2018012,2021-02-18 23:45:00,FR*V75*EBELI*96*1,2.0,1.0,0.0,0.0,95,5,22202,48.835499,2.349650,75013,south
2018013,2021-02-18 23:45:00,FR*V75*EBELI*97*1,1.0,1.0,1.0,0.0,95,5,22202,48.839947,2.397822,75012,east
2018014,2021-02-18 23:45:00,FR*V75*EBELI*98*1,2.0,0.0,1.0,0.0,95,5,22202,48.849980,2.361920,75004,south


In [ ]:

to_remove=[]#to be removed all totatl lists of tode_ list
to_be_fill_by_back=[]# for back fill indexes
to_be_fill_by_front=[]
all_the_null=df[(df.Available.isnull())].index.copy()

def get_index_to_fill(data,fill,n):
    list_=data[(data.Available.isnull())].index.copy()#find index null
    rr=data[["Available"]].fillna(method=fill, axis=0, limit=n)# fill by index
    symmertic_data=list_.symmetric_difference(rr[rr.Available.isnull()].index)# all index that we can get by back fill
    #if fill=="ffill" :print("There is ",len(list_),"missing values.")
    TO_=data[data.index.isin(symmertic_data)]["index"].values#real index to be fill by back fill
    return  TO_

span=4
for sta in df.Station.unique():
    #backword data set 
    df202=df.copy()
    df202=df202.reindex(index=df202.index[::-1])
    #forward data set
    test_df=df[df.Station==sta].copy().reset_index()#by station and creat save orginal index in index
    TO_back= get_index_to_fill(test_df,fill="bfill",n=8)
    TO_front=get_index_to_fill(test_df,fill="ffill",n=8)
    to_be_fill_by_back.append(TO_back)
    to_be_fill_by_front.append(TO_front)
   # print("We will recover only",len(TO_frontEWE)+len(TO_backEWE))
    #forward fill
    df.loc[df.Station==sta,["Available_f_m","Charging_f_m","Passive_f_m",
       "Other_f_m"]]=df.loc[df.Station==sta,["Available","Charging","Passive","Other"]].ewm(span=span).mean().values

    df202.loc[df202.Station==sta,["Available_f_m2","Charging_f_m2","Passive_f_m2",
      "Other_f_m2"]]=df202.loc[df202.Station==sta,["Available","Charging","Passive","Other"]].ewm(span=span).mean().values
df202=df202.reindex(index=df202.index[::-1])

#removing only index that were not fill by bfill and ffill
to_remove=np.setdiff1d(all_the_null,np.append(np.concatenate(to_be_fill_by_back)
            ,np.concatenate(to_be_fill_by_front)))




#keep the back fill in fron fill
to_be_fill_by_back33=np.concatenate(to_be_fill_by_back)
df.loc[df.index.isin(to_be_fill_by_back33),["Available_f_m","Charging_f_m","Passive_f_m",
       "Other_f_m"]]=df202.loc[df202.index.isin(to_be_fill_by_back33),["Available_f_m2","Charging_f_m2","Passive_f_m2",
       "Other_f_m2"]].values



# get beck the good values that already were there and that it got affacted by EWM
df5=df.copy()
print(train_station.shape)
for tar in ["Available","Charging","Passive","Other"]:
  df5[tar]=np.rint(np.where(df[tar].isnull(), df[tar+"_f_m"],df[tar])).astype(int)
print(df5.shape)
df5.tail()
#drop values not needed
df5=df5.drop(df5.index[to_remove])
print(df5.shape)#(1855206, 17) 2018016

(1851083, 13)
(2018016, 17)
(1857413, 17)


remove value bigger or smaller than 3

In [ ]:
df6=df5[((df5[["Available","Charging","Passive","Other"]].sum(axis=1)>3)|(
    df5[["Available","Charging","Passive","Other"]].sum(axis=1)<3))==False].reset_index(drop=1).copy()

df5.shape[0]-df6.shape[0]



300

remove value that are equal 

In [ ]:
state="Charging"
df7=df6[["date" ,"Station",state]].pivot(index='date', columns='Station', values=state).copy()
index_rem=df7[df7.eq(df7.iloc[:, 5], axis=0).all(1)].index
index_rem
df11=df6[df6.date.isin(index_rem)==False].copy()

DatetimeIndex(['2020-07-03 17:00:00', '2020-07-03 17:15:00',
               '2020-07-03 17:30:00', '2020-07-03 17:45:00',
               '2020-07-03 18:00:00', '2020-07-03 18:15:00',
               '2020-07-03 18:30:00', '2020-07-03 18:45:00',
               '2020-07-03 19:00:00', '2020-07-04 08:00:00',
               ...
               '2020-09-23 16:45:00', '2020-09-23 17:00:00',
               '2020-09-23 17:15:00', '2020-09-23 17:30:00',
               '2020-09-23 17:45:00', '2020-09-23 18:00:00',
               '2020-09-23 18:15:00', '2020-09-23 18:30:00',
               '2020-09-23 18:45:00', '2020-09-23 19:00:00'],
              dtype='datetime64[ns]', name='date', length=718, freq=None)

In [ ]:
state="Passive"
df8=df6[["date" ,"Station",state]].pivot(index='date', columns='Station', values=state).copy()
df8[df8.eq(df8.iloc[:, 5], axis=0).all(1)].head()

Station,FR*V75*EBELI*1*1,FR*V75*EBELI*10*1,FR*V75*EBELI*11*1,FR*V75*EBELI*12*1,FR*V75*EBELI*13*1,FR*V75*EBELI*14*1,FR*V75*EBELI*15*1,FR*V75*EBELI*16*1,FR*V75*EBELI*17*1,FR*V75*EBELI*18*1,...,FR*V75*EBELI*9*1,FR*V75*EBELI*91*1,FR*V75*EBELI*92*1,FR*V75*EBELI*93*1,FR*V75*EBELI*94*1,FR*V75*EBELI*95*1,FR*V75*EBELI*96*1,FR*V75*EBELI*97*1,FR*V75*EBELI*98*1,FR*V75*EBELI*99*1
date,,,,,,,,,,,,,,,,,,,,,


In [ ]:
np.unique(df6.Available.astype(str) + df6.Charging.astype(str) + df6.Passive.astype(str)) 

array(['000', '001', '002', '003', '010', '011', '012', '020', '021',
       '030', '100', '101', '102', '110', '111', '120', '200', '201',
       '210', '300'], dtype=object)

In [ ]:
index_rem

Station,FR*V75*EBELI*1*1,FR*V75*EBELI*10*1,FR*V75*EBELI*11*1,FR*V75*EBELI*12*1,FR*V75*EBELI*13*1,FR*V75*EBELI*14*1,FR*V75*EBELI*15*1,FR*V75*EBELI*16*1,FR*V75*EBELI*17*1,FR*V75*EBELI*18*1,...,FR*V75*EBELI*9*1,FR*V75*EBELI*91*1,FR*V75*EBELI*92*1,FR*V75*EBELI*93*1,FR*V75*EBELI*94*1,FR*V75*EBELI*95*1,FR*V75*EBELI*96*1,FR*V75*EBELI*97*1,FR*V75*EBELI*98*1,FR*V75*EBELI*99*1
date,,,,,,,,,,,,,,,,,,,,,
2020-07-03 17:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-07-03 17:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-07-03 17:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-07-03 17:45:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-07-03 18:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-23 18:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-09-23 18:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-09-23 18:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df11=df6[df6.date.isin(index_rem)==False].copy()

In [ ]:
df11["lable"]=df11.Available.astype(int).astype(str) + df11.Charging.astype(int).astype(str) + df11.Passive.astype(int).astype(str)+ df11.Other.astype(int).astype(str)
df11.columns

Index(['date', 'Station', 'Available', 'Charging', 'Passive', 'Other', 'tod',
       'dow', 'trend', 'Latitude', 'Longitude', 'Postcode', 'area',
       'Available_f_m', 'Charging_f_m', 'Passive_f_m', 'Other_f_m', 'lable'],
      dtype='object')

In [ ]:
df11[['date', 'Station', 'Available', 'Charging', 'Passive', 'Other', 'tod',
       'dow', 'trend', 'Latitude', 'Longitude', 'Postcode', 'area', 'lable']].reset_index(drop=1).to_csv("train_onlyNext4EWM4andBack_EWM_remChar.csv",index=False)

In [ ]:
df6[df6.Available.isnull()]

,date,Station,Available,Charging,Passive,Other,tod,dow,trend,Latitude,Longitude,Postcode,area,Available_f_m,Charging_f_m,Passive_f_m,Other_f_m,lable


In [ ]:
#remCharEWM4.csv

In [ ]:
# 

# Making remCharEWM4.csv


In [ ]:
train_station = pd.read_csv("data/train.csv", sep=",")
train_station['date'] = pd.to_datetime(train_station['date'])
train_station['Postcode'] = train_station['Postcode'].astype(str)


state="Charging"
df7=train_station[["date" ,"Station",state]].pivot(index='date', columns='Station', values=state).copy()
index_rem=df7[df7.eq(df7.iloc[:, 5], axis=0).all(1)].index
index_rem
train_station=train_station[train_station.date.isin(index_rem)==False].copy()

In [ ]:
for sta in df.Station.unique():
  df.loc[df.Station==sta,["Available_f_m","Charging_f_m","Passive_f_m",
        "Other_f_m"]]=df.loc[df.Station==sta,["Available","Charging","Passive","Other"]].ewm(span=4).mean().values

In [ ]:
df5=df.copy()
print(train_station.shape)
df5.Available=(np.where(df["Available"].isnull(), df["Available_f_m"],df["Available"]))
df5.Charging=(np.where(df["Charging"].isnull(), df["Charging_f_m"],df["Charging"]))
df5.Passive=(np.where(df["Passive"].isnull(), df["Passive_f_m"],df["Passive"]))
df5.Other=(np.where(df["Other"].isnull(), df["Other_f_m"],df["Other"]))

(1785745, 13)


In [ ]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2018016 entries, 0 to 2018015
Data columns (total 17 columns):
 #   Column         Dtype         
---  ------         -----         
 0   date           datetime64[ns]
 1   Station        object        
 2   Available      float64       
 3   Charging       float64       
 4   Passive        float64       
 5   Other          float64       
 6   tod            int64         
 7   dow            int64         
 8   trend          int64         
 9   Latitude       float64       
 10  Longitude      float64       
 11  Postcode       object        
 12  area           object        
 13  Available_f_m  float64       
 14  Charging_f_m   float64       
 15  Passive_f_m    float64       
 16  Other_f_m      float64       
dtypes: datetime64[ns](1), float64(10), int64(3), object(3)
memory usage: 277.1+ MB


In [ ]:
df6=df5[((df5[["Available","Charging","Passive","Other"]].sum(axis=1)>3.1)|(
    df5[["Available","Charging","Passive","Other"]].sum(axis=1)<2))==False].reset_index(drop=1).copy()

(df5.shape[0])-df6.shape[0]

49453

In [ ]:
df5[['date', 'Station', 'Available', 'Charging', 'Passive', 'Other', 'tod',
       'dow', 'trend', 'Latitude', 'Longitude', 'Postcode', 'area']].reset_index(drop=1).to_csv("data/remCharEWM4.csv",index=False)